In [1]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from collections import Counter
from src.utils import import_dta, import_tracks_dta, successful_creators_followers,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import numpy as np
import datetime
import pandas as pd
from tqdm import tqdm
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#General options

path_dir = r"/Users/../Volumes/Raw/"
music_before = 30
low_success = 0.5
high_success = 0.9
rank_percentiles = False

In [3]:
def import_indegree_dask(path='/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/'):
    df = pd.read_pickle('{}df0.pkl'.format(path))
    df1 = pd.read_pickle('{}df1.pkl'.format(path))
    df2 = pd.read_pickle('{}df2.pkl'.format(path))
    df3 = pd.read_pickle('{}df3.pkl'.format(path))
    df4 = pd.read_pickle('{}df4.pkl'.format(path))
    df5 = pd.read_pickle('{}df5.pkl'.format(path))
    df6 = pd.read_pickle('{}df6.pkl'.format(path))
    df7 = pd.read_pickle('{}df7.pkl'.format(path))
    df8 = pd.read_pickle('{}df8.pkl'.format(path))
    df9 = pd.read_pickle('{}df9.pkl'.format(path))
    
    ddf = dd.from_pandas(df, npartitions = 3)
    ddf1 = dd.from_pandas(df1, npartitions = 3)
    ddf2 = dd.from_pandas(df2, npartitions = 3)
    ddf3 = dd.from_pandas(df3, npartitions = 3)
    ddf4 = dd.from_pandas(df4, npartitions = 3)
    ddf5 = dd.from_pandas(df5, npartitions = 3)
    ddf6 = dd.from_pandas(df6, npartitions = 3)
    ddf7 = dd.from_pandas(df7, npartitions = 3)
    ddf8 = dd.from_pandas(df8, npartitions = 3)
    ddf9 = dd.from_pandas(df9, npartitions = 3)
    
    concatdf = dd.multi.concat([ddf,ddf1,ddf2,ddf3,ddf4,ddf5,ddf6,ddf7,ddf8,ddf9])
    
    return concatdf

In [4]:
def process_date(date):
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.datetime.strptime(date, "%Y-%W-%w")
    return dt

In [5]:
#Creator's Actions data

In [6]:
!ls /Users/../Volumes/Track_plays

12sample_plays_made.dta


In [7]:
#we will skip comments for now

#affiliations :follows
#favoritings :likes

#used in filtering:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

#used to compute creato_actions
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

#Used to track information on the 1st degree connections
user_info_1st = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_1st.columns = ['user_id', 'type', 'entered_platform'];
user_info = import_dta(path_dir, "12sample_user_infos.dta");

#Used to compute creator's success measure
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

#previously used to filter creators
tracks_first_month = sample_creators_music(tracks, music_before)# music_before changed so it 
#suffices to have made a track available
mask = tracks_first_month.track_available == 1
tracks_first_month = tracks_first_month[mask]

mask = follows_received.contact_id.isin(tracks_first_month.user_id)
follows_received_30 = follows_received[mask]
followers_per_creator_60_static = successful_creators_followers(follows_received_30, 60, 
                                                        perc1 = low_success, perc2 = high_success)

mask = user_info.type == 'creator'
creator_ids = user_info[mask].user_id.unique()

mask = follows_received.contact_id.isin(creator_ids)
follows_received_creators = follows_received[mask]

followers_per_creator_60_static = successful_creators_followers(follows_received_creators, 60, 
                                                        perc1 = low_success, perc2 = high_success, subset_creators = tracks_first_month)

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)
2018-02-03 00:00:00
High influencer boundary: 66.0
Low influencer boundary: 11.0
2018-02-03 00:00:00
High influencer boundary: 53.0
Low influencer boundary: 6.0


In [9]:
type(tracks_first_month)== pd.DataFrame

True

In [10]:
#successful_user_ids
mask = followers_per_creator_60_static.high_low == 'low'
unsuccessful_ids = followers_per_creator_60_static.loc[mask].user_id.unique()

mask = followers_per_creator_60_static.high_low == 'high'
successful_ids = followers_per_creator_60_static.loc[mask].user_id.unique()

In [11]:
# Aggregates preprocessed outdegree of 1st degree users

d = {}
for i in range(6):
   d[str(i)] = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/{}.pkl'.format(i)) 
   d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])
    
data_out = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])
data_out.set_index('created_at', inplace = True)

In [12]:
data_in = import_indegree_dask()

In [13]:
#Generates a data sent with all messages sent by creators

actions_sent = gen_outbound_creators(follows_sent, shares_sent, likes_sent, comments_sent, tracks,
                                     messages_sent = messages_sent, filter_creators = True)
#mask = actions_sent.user_id.isin(tracks_first_month.user_id.unique())
#actions_sent = actions_sent[mask]

In [14]:
#filter only outbound activities before the follow is received
print('Obtaining fanbase...')
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follower_list(follows_received)
actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_followers =  actions_sent[mask]
actions_sent_non_followers['week_yr_date'] = actions_sent_non_followers.week_yr.apply(lambda x: process_date(x))

Obtaining fanbase...


/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_84193/2662936126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_followers['week_yr_date'] = actions_sent_non_followers.week_yr.apply(lambda x: process_date(x))


In [15]:
mask = actions_sent_non_followers.user_id.isin(user_info.loc[user_info.type == 'creator'].user_id.unique())
actions_sent_non_followers = actions_sent_non_followers[mask]

In [16]:
class MultiArmedBandits:
    
    def __init__(self, arm_names, user_id):
        self.user_id = user_id
        self.arm_names = arm_names
        self.arms = {k:{'Sent':0,'Reward':0} for k in arm_names}
        self.increment = {k:{'Sent':0,'Reward':0} for k in arm_names}
        self.means = np.array(len(arm_names))
        self.variances = np.array(len(arm_names))
        self.last_arm_pulled = None
        self.arm_switches = 0
        self.total_trials = 0 
        self.trials_at_day = 0
        self.rewards = 0
    
    def initialize_priors(self):
        pass
    
    def update_increment(self, target, outcome):
        self.increment[target]['Sent'] += 1
        self.increment[target]['Reward'] += outcome
        self.arm_switches += self.arm_switches + (self.last_arm_pulled != str(target))
        self.last_arm_pulled = str(target)
        self.total_trials += 1
        self.trials_at_day += 1
        self.rewards += outcome
    
    def update_arms(self):
        for k in self.increment.keys():
            for j in self.increment[k]:
                self.arms[k][j] += self.increment[k][j]
        self.increment = {k:{'Sent':0,'Reward':0} for k in self.arm_names}
        self.trials_at_day = 0
                
    def estimate_moments(self):
        
        Sent = np.array([a[1]['Sent'] for a in self.arms.items()])
        Reward = np.array([a[1]['Reward'] for a in self.arms.items()])
        
        alpha = Reward + 1 
        beta = (Sent-Reward)+1

        mean = alpha/(alpha+beta)
        variance = (alpha*beta)/((alpha+beta+1)*((alpha+beta)**2))
        
        self.means = np.transpose(np.around(mean,3))
        self.variances = np.transpose(np.around(np.sqrt(variance),3))

To do:
- Label data -done
- Mock classification rule (keep previous)-done
- Function to count outdegree/indegree using increments/ Obs: not necessary since we are using daily batches.

In [17]:
def create_target(data_actions_sent, interval):
    # Target Creation
    delta = datetime.timedelta(days = interval)
    mask = (data_actions_sent['follower_since'] <= (data_actions_sent['date_sent'] + delta).dt.floor('d'))

    data_actions_sent.loc[mask, 'reward'] = 1
    mask = data_actions_sent['reward'].isnull()
    data_actions_sent.loc[mask, 'reward'] = 0
    return data_actions_sent

In [18]:
def update_membership_table(data, upper_limit, d_percentiles = 10):
    
    data = data[data.created_at<upper_limit]
    data = data.groupby('contact_id').agg({'size':'sum'})
    steps = 100//d_percentiles
    
    #merge with user info to obtain users that are not followed by anyone at the current date
    data = user_info_1st.merge(data, left_on = 'user_id', right_on = 'contact_id', how= 'outer')
    data.loc[data['size'].isnull(), 'size'] = 0
    data = data[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    #filter out users that didnt exist in the current date
    mask = data['entered_platform'] <= upper_limit
    data = data.loc[mask]
    
    mask = (data['size']>0)
    data.loc[~mask, 'type'] = 0 
    
    cutpoints = np.percentile(data[mask]['size'], np.arange(0,100, steps)) 
    
    for i in range(len(cutpoints)):
        low = cutpoints[i]
        try:
            high = cutpoints[i+1]
        except:
            high = 100000
        
        mask2 = (data[mask]['size']>=low) & (data[mask]['size'] <high) & (data['size']>0)
        data.loc[mask2,'type'] = i + 1 
    return data

In [19]:
def update_membership_table_dask(data, upper_limit, user_info = user_info_1st, d_percentiles = 10):
    
    data = data[data.created_at.dt.floor('d')<=upper_limit]
    data = data.groupby('contact_id').agg({'size':'sum'}).compute()
    steps = 100//d_percentiles
    
    #merge with user info to obtain users that are not followed by anyone at the current date
    data = user_info_1st.merge(data, left_on = 'user_id', right_on = 'contact_id', how= 'outer')
    data.loc[data['size'].isnull(), 'size'] = 0
    data = data[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    #filter out users that didnt exist in the current date
    mask = data['entered_platform'].dt.floor('d') <= upper_limit
    data = data.loc[mask]
    
    mask = (data['size']>0)
    data.loc[~mask, 'type'] = 0 
    
    #cutpoints ignore users with 0 followers as they will appear in their own category
    #cutpoints = np.percentile(data[mask]['size'], np.arange(1,100, steps)) 
    cutpoints = np.percentile(data['size'], np.arange(0,100, steps)) 

    for i in range(len(cutpoints)):
        low = cutpoints[i]
        try:
            high = cutpoints[i+1]
        except:
            high = 100000
        
        mask2 = (data[mask]['size']>=low) & (data[mask]['size'] <high) & (data['size']>0)
        data.loc[mask2,'type'] = i + 1 
    return data

In [20]:
arm_names = [i for i in range(0,11)]
labeled_dateset = create_target(actions_sent_non_followers,1)
mask = (actions_sent_non_followers.user_id.isin(successful_ids) | actions_sent_non_followers.user_id.isin(unsuccessful_ids))
df = actions_sent_non_followers[mask]
d = {}
e = {}
first_day = min(actions_sent_non_followers.date_sent.dt.floor('d').unique())
last_day =  max(actions_sent_non_followers.date_sent.dt.floor('d'))
membership = update_membership_table_dask(data_in, first_day, d_percentiles = 10)
fan_not_found = []
user_ids = df.user_id.unique()

#initialize arms
for user_id in df.user_id.unique():
    d[user_id] = MultiArmedBandits(arm_names, user_id)

    dataset_mean = pd.DataFrame({**{'date':pd.date_range(start = first_day, end=last_day)},
                                 **{str(k):0 for k in range(0,11)}}) 
    dataset_std = pd.DataFrame({**{'date':pd.date_range(start = first_day, end=last_day)},
                                 **{str(k):0 for k in range(0,11)}}) 
    trials = pd.DataFrame({**{'date':pd.date_range(start = first_day, end=last_day)},
                                 **{'trials_at_date':np.nan}}) 
    
    e[user_id] = {'dataset_std': dataset_std, 'dataset_mean': dataset_mean, 'trials' : trials}

#update arms
for day in tqdm(actions_sent_non_followers.date_sent.dt.floor('d').unique()):
    for user_id in df.loc[df['date_sent'].dt.floor('d') == day].user_id.unique():
        df_subset = df.loc[(df['date_sent'].dt.floor('d') == day)&(df['user_id'] == user_id)]
        for action in range(len(df_subset)):
            fan_id = df_subset.iloc[action].fan_id
            try:
                user_type = membership.loc[membership.index == fan_id].type.values[0]
                d[user_id].update_increment(user_type, df_subset.iloc[action].reward)
            except KeyError:
                fan_not_found.append([fan_id, day, df_subset.iloc[action].outbound_activity])
            except IndexError:
                fan_not_found.append([fan_id, day, df_subset.iloc[action].outbound_activity])
  #REWRITE THIS AS A LOOP OVER ALL USER_IDS          
    for user_id in user_ids:
        e[user_id]['trials'].loc[e[user_id]['trials'].date == day, 'trials_at_date'] = d[user_id].trials_at_day
        d[user_id].update_arms()
        d[user_id].estimate_moments()
        e[user_id]['dataset_mean'].loc[e[user_id]['dataset_mean'].date == day, '0':] = np.transpose(d[user_id].means)
        e[user_id]['dataset_std'].loc[e[user_id]['dataset_mean'].date == day, '0':] = np.transpose(d[user_id].variances)
        
    membership = update_membership_table_dask(data_in, day, d_percentiles = 10)
    
for user_id in df.user_id.unique():
    e[user_id]['total_trials'] = d[user_id].total_trials

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 864/864 [2:08:00<00:00,  8.89s/it]


In [21]:
def weekly_membership(df, data_in):

    df_copy = df.copy()
    df_copy['type'] = 0
    
    first_date = min(df_copy.date_sent)
    membership = update_membership_table_dask(data_in, first_day, d_percentiles = 10)
    
    for day in tqdm(df_copy.date_sent.dt.floor('d').unique()):
        mask = (df_copy['date_sent'].dt.floor('d') == day)
        df_subset = df_copy[mask]
        for i in range(df_copy[mask].shape[0]):
            fan_id = df_subset.iloc[i].fan_id
            try:
                df_subset.iloc[i,-1] = (int(membership.loc[membership.index == fan_id]['type']))
                print('.', end='')
        
            except:
                
                df_subset.iloc[i,-1] = 'Fan not found'
                print('x', end ='')
                
        df_copy[mask] = df_subset
        membership = update_membership_table_dask(data_in, day, d_percentiles = 10)

    return df_copy

In [22]:
dynamics

NameError: name 'dynamics' is not defined

In [ ]:
def plot_learning(data, number, title):
    
    
    data_grouped = data.groupby('week_yr_date', as_index = False).size()
    data.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity',
       'interaction_week', 'interaction_year', 'week_yr',
       'follower_since', 'week_yr_date', 'reward',
       'Receiver Type']
    
    lim_inf = min(data.date_sent)
    lim_sup = max(data.date_sent)
    
    fig, ax = plt.subplots(figsize=(15, 9))

    hue_order_l = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

    sns.set(rc={'figure.figsize':(12,8)})
    sns.set_style("white")
    sns.kdeplot(data=data,
            x="week_yr_date",
            hue='Receiver Type',
            hue_order= hue_order_l,
            multiple="fill",
            #weights="Val",
            bw_adjust = 1)\
    .set(xlim=(lim_inf, lim_sup), title='Figure {}: {} - Share of trials per receiver type.'.format(number, title), xlabel='Week',
     ylabel='Share')
    
    ax2 = plt.twinx()
    sns.lineplot(data=data_grouped,
                 x = "week_yr_date", 
                 y = "size", 
                 color="black", 
                 ax=ax2, 
                 legend = 'auto',
                 linewidth = 2)\
    .set(ylabel='Total Promotional Actions', yscale="log")
    plt.annotate('''Note: each color corresponds to a receiver type. The black line is the total number of actions per week.'''
             , (0,0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top', fontsize=14);

    #ax.legend(title='Receiver type', loc='best', labels = ['10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']);

In [ ]:
mask = (actions_sent_non_followers.user_id.isin(successful_ids) | actions_sent_non_followers.user_id.isin(unsuccessful_ids))
df = actions_sent_non_followers[mask]
#dynamics = weekly_membership(df, data_in)

In [ ]:
mask = (dynamics.user_id.isin(successful_ids)) & (dynamics['Receiver Type'] != 'Fan not found')
success_dynamics = dynamics[mask]
plot_learning(success_dynamics, 21, 'Successful')

In [ ]:
mask = (dynamics.user_id.isin(unsuccessful_ids)) & (dynamics['Receiver Type'] != 'Fan not found')
unsuccess_dynamics = dynamics[mask]
plot_learning(unsuccess_dynamics, 22, 'Unsuccessful')

Draft of function to calculate estimates:

-estimates: take estimates at last day
-trials: total, median, mean and std|get from arms
-rewards: get from arms

In [ ]:
#one for successful and one for unsuccessful?
mask = (actions_sent_non_followers.user_id.isin(successful_ids) | actions_sent_non_followers.user_id.isin(unsuccessful_ids))
df = actions_sent_non_followers[mask]

dataset_last_mean = pd.DataFrame(index = df.user_id.unique(), columns = [str(i) for i in range(0,11)])
dataset_last_std = pd.DataFrame(index = df.user_id.unique(), columns = [str(i) for i in range(0,11)])
df_trial_info = pd.DataFrame(index = df.user_id.unique(), columns = ['total_trials'])
df_summary_rewards = pd.DataFrame(index = df.user_id.unique(), columns = ['Rewards'])
arm_names = [str(i) for i in range(0,11)]

for user_id in actions_sent_non_followers.user_id.unique():
    try:
        dataset_last_mean.loc[user_id, :] = e[user_id]['dataset_mean'].iloc[-1]
        dataset_last_std.loc[user_id, :] = e[user_id]['dataset_std'].iloc[-1]
        df_trial_info.loc[user_id, :] = d[user_id].total_trials
        df_summary_rewards.loc[user_id, :] = d[user_id].rewards
    except:
        pass
    

init = 0

df_trial_info = pd.DataFrame(index = ["{:d}".format(x) for x in range(init,11)] , columns = ['Unique_creators', 'trials'])
for j in range(init,11):
    df_trial_info.iloc[j-init]['trials'] = list()
    df_trial_info.iloc[j-init]['Unique_creators'] = 0
for key in d.keys():
    for j in range(init,11):
        if d[key].arms[j]['Sent'] != 0:
            df_trial_info.iloc[j-init]['Unique_creators'] = df_trial_info.iloc[j-init]['Unique_creators']+1
        df_trial_info.iloc[j-init]['trials'].extend([d[key].arms[j]['Sent']])
df_trial_info['Median_trials'] = df_trial_info.trials.apply(lambda x: np.median(x))
df_trial_info['Mean_trials'] = df_trial_info.trials.apply(lambda x: np.mean(x))
df_trial_info['Std_trials'] = df_trial_info.trials.apply(lambda x: np.std(x))

summary = pd.DataFrame(dataset_last_mean[arm_names].mean(), columns = ['Mean Estimate']).merge(
        pd.DataFrame(dataset_last_std[arm_names].mean(), columns = ['Std Estimate']), left_index=True, right_index=True).merge(
    df_trial_info[['Median_trials', 'Mean_trials', 'Std_trials']
        ],left_index=True, right_index=True).reindex(arm_names)
    

In [ ]:
def gen_estimates(df, arm_names):

    dataset_last_mean = pd.DataFrame(index = df.user_id.unique(), columns = arm_names)
    dataset_last_std = pd.DataFrame(index = df.user_id.unique(), columns = arm_names)
    df_trial_info = pd.DataFrame(index = df.user_id.unique(), columns = ['total_trials'])
    df_summary_rewards = pd.DataFrame(index = df.user_id.unique(), columns = ['Rewards'])
    arm_names = [i for i in range(0,11)]

    for user_id in df.user_id.unique():
        try:
            dataset_last_mean.loc[user_id, :] = list(e[user_id]['dataset_mean'].iloc[-1,1:])
            dataset_last_std.loc[user_id, :] = list(e[user_id]['dataset_std'].iloc[-1,1:])
            df_trial_info.loc[user_id, :] = d[user_id].total_trials
            df_summary_rewards.loc[user_id, :] = d[user_id].rewards
        except:
            pass

    init = 0
      
    df_trial_info = pd.DataFrame(index = arm_names , columns = ['Rewards', 'Unique_creators', 'trials'])
    for j in range(len(arm_names)):
        df_trial_info.iloc[j-init]['trials'] = list()
        df_trial_info.iloc[j-init]['Unique_creators'] = 0
        df_trial_info.iloc[j-init]['Rewards'] = 0
    for key in df.user_id.unique():
        for j in range(len(arm_names)):
            df_trial_info.iloc[j-init]['Rewards'] += d[key].arms[j]['Reward']
            if d[key].arms[j]['Sent'] != 0:
                df_trial_info.iloc[j-init]['Unique_creators'] = df_trial_info.iloc[j-init]['Unique_creators']+1
            df_trial_info.iloc[j-init]['trials'].extend([d[key].arms[j]['Sent']])
    df_trial_info['Total_trials'] = df_trial_info.trials.apply(lambda x: np.sum(x))
    df_trial_info['Median_trials'] = df_trial_info.trials.apply(lambda x: np.median(x))
    df_trial_info['Mean_trials'] = df_trial_info.trials.apply(lambda x: np.mean(x))
    df_trial_info['Std_trials'] = df_trial_info.trials.apply(lambda x: np.std(x))

    summary = pd.DataFrame(dataset_last_mean[arm_names].mean(), columns = ['Mean Estimate']).merge(
            pd.DataFrame(dataset_last_std[arm_names].mean(), columns = ['Std Estimate']), left_index=True, right_index=True).merge(
        df_trial_info[['Rewards','Unique_creators','Total_trials','Median_trials', 'Mean_trials', 'Std_trials']
            ],left_index=True, right_index=True).reindex(arm_names)
    
    return summary

In [ ]:
mask = (actions_sent_non_followers.user_id.isin(successful_ids))
df = actions_sent_non_followers[mask]
print('Unique successful creators:', df.user_id.nunique())
successful_summary = gen_estimates(df, arm_names)

In [ ]:
successful_summary

In [ ]:
successful_summary.to_excel('tables/sucessful.xlsx')

In [ ]:
mask = (actions_sent_non_followers.user_id.isin(unsuccessful_ids))
df = actions_sent_non_followers[mask]
print('Unique unsuccessful creators:', df.user_id.nunique())
unsuccessful_summary = gen_estimates(df, arm_names)

In [ ]:
!ls

In [ ]:
unsuccessful_summary.to_excel('tables/unsucessful.xlsx')

In [ ]:
tracks_first_month_star = sample_creators_music(tracks, 60)# music_before changed so it 
#suffices to have made a track available
mask = tracks_first_month_star.track_available == 1
tracks_first_month_star = tracks_first_month_star[mask]

In [ ]:
tracks_first_month_star.user_id.nunique()

In [ ]:
tracks_first_month.user_id.nunique()

In [ ]:
tracks_first_month_star.user_id.nunique()/4604